# CPSC 330 - Applied Machine Learning 

## Homework 8: Time series

### Associated lectures: [Lectures 19 and 20](https://github.com/UBC-CS/cpsc330-2024s/tree/main/lectures)

**See PrairieLearn for _due date_ and _submission_**

## Imports

In [1]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size $\leq$ 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024s/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [2]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [3]:
df.shape

(18249, 13)

In [4]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [5]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [6]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [7]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 

_Points:_ 4

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture, we had different measurements for each Location. How about this avocado sales dataset? For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

The categorical features that provide separate measurements within each given `Date` are `type` and `region`, as `year` is directly derived from `Date`.

In [8]:
display(df_train.loc[df_train['Date'] == '2015-01-04 00:00:00'].head())
display(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['type'] == 'conventional')].head())
display(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['region'] == "Albany")].head())
display(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['type'] == 'conventional') & (df_train['region'] == "Albany")].head())

print(df_train['type'].unique())
print(df_train.loc[df_train['Date'] == '2015-01-04 00:00:00'].shape)
print(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['type'] == 'conventional')].shape)
print(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['region'] == "Albany")].shape)
print(df_train.loc[(df_train['Date'] == '2015-01-04 00:00:00') & (df_train['type'] == 'conventional') & (df_train['region'] == "Albany")].shape)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
51,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta
51,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,BaltimoreWashington
51,2015-01-04,1.01,80034.32,44562.12,24964.23,2752.35,7755.62,6064.30,1691.32,0.0,conventional,2015,Boise
51,2015-01-04,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015,Boston


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
51,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta
51,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,BaltimoreWashington
51,2015-01-04,1.01,80034.32,44562.12,24964.23,2752.35,7755.62,6064.30,1691.32,0.0,conventional,2015,Boise
51,2015-01-04,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015,Boston


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.9,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
51,2015-01-04,1.79,1373.95,57.42,153.88,0.0,1162.65,1162.65,0.00,0.0,organic,2015,Albany


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.5,28287.42,49.9,9716.46,9186.93,529.53,0.0,conventional,2015,Albany


['conventional' 'organic']
(108, 13)
(54, 13)
(2, 13)
(1, 13)


In [9]:
# df_train_sort = df_train.sort_values(by=["region", "type", "Date"])
# df_test_sort = df_test.sort_values(by=["region", "type", "Date"])
# display(df_train_sort.head())

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 

_Points:_ 4

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

The typical spacing of measurements within the dataset appears to be once every 7 days, where there would have been approximately 142 weeks in the time frame for the training dataset. The only exception is `region == "WestTexNewMexico" & type == "organic"`, having 1 measurement for 14 days and 21 days, as well as missing 3 measurements overall.

142 - (137 + 1 + 1) = 3

In [10]:
# Adapted from Lectue 19
# count = 0 
dict = {}
for name, group in df_train.groupby(["region", "type"]):
    dict[name] = group["Date"].sort_values().diff().value_counts()
    # print("%-30s %s" % (name, group["Date"].sort_values().diff().value_counts()))
    # print('\n\n\n')
    # count+=1
    # if count == 2: 
    #     break

resultstable = pd.DataFrame(dict).T
display(resultstable)

Date                              7 days  14 days  21 days
Albany              conventional   142.0      NaN      NaN
                    organic        142.0      NaN      NaN
Atlanta             conventional   142.0      NaN      NaN
                    organic        142.0      NaN      NaN
BaltimoreWashington conventional   142.0      NaN      NaN
...                                  ...      ...      ...
TotalUS             organic        142.0      NaN      NaN
West                conventional   142.0      NaN      NaN
                    organic        142.0      NaN      NaN
WestTexNewMexico    conventional   142.0      NaN      NaN
                    organic        137.0      1.0      1.0

[108 rows x 3 columns]

In [11]:
print(resultstable["7 days"].unique())
print(resultstable["14 days"].unique())
print(resultstable["21 days"].unique())

[142. 137.]
[nan  1.]
[nan  1.]


In [12]:
display(resultstable.loc[resultstable['7 days'] != 142])

,Date,7 days,14 days,21 days
WestTexNewMexico,organic,137.0,1.0,1.0


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 

_Points:_ 4

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

https://en.wikipedia.org/wiki/Southeastern_United_States

There may be some overlapping regions, as several of the regions encompass larger geographical regions while others consist of specific cities. For instance, `Southeast` refers to the American Southeast, which includes the locations `Nashville`, `Louisville` and `BaltimoreWashington`.

In [13]:
df_train["region"].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

EDIT SUGGESTION:
- Count number of timestamps and see if they add up to the same as `TotalUS`

In [14]:
ts = pd.Timestamp(2015, 1, 4)
print(df_train.query("region == 'TotalUS' and type == 'conventional' and Date == @ts")["Total Volume"].values[0])
ts = pd.Timestamp(2015, 1, 4)
print(df_train.query("region != 'TotalUS' and type == 'conventional' and Date == @ts")["Total Volume"].sum())

31324277.73
51730521.73


<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/lectures/mathias/19_time-series.ipynb), with some improvements.

In [15]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [16]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [17]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [18]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

### 1.4 `AveragePrice` baseline 

_Points:_ 4

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

Given the baseline approach, there is a train $R^2$ score of 0.8285800937261841, and a test $R^2$ score of 0.7631780188583048.

In [19]:
train_r2 = r2_score(df_train["AveragePriceNextWeek"], df_train["AveragePrice"]) 
test_r2 = r2_score(df_test["AveragePriceNextWeek"], df_test["AveragePrice"]) 
print("train_r2: " + str(train_r2))
print("test_r2: " + str(test_r2))

train_r2: 0.8285800937261841
test_r2: 0.7631780188583048


<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price

_Points:_ 10

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. We got to 0.80, but not beyond that. Let us know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

There are 4 options available for encoding the date: 
- Time as a number (Assuming that time in general dictates the forecasting)
- One-hot encoding of year (Assuming that the year dictates the forecasting)
- One-hot encoding of month (Assuming that the month dictates the forecasting)
- Time as a number of weeks (Assuming that time in general dictates the forecasting)

For the purposes of this model, `year` is treated as a categorical variable, though it will be removed for the first and third options.

EDIT SUGGESTIONS: 
- `Total Bags` is approximately equal to the sum of `Small Bags`, `Large Bags` and `XLarge Bags` and thus should be removed.
- `Total Volume` is not approximately equal to the sum of the 3 avocado types and thus should be retained.
- No null values, so no imputation is required.
- Plot time series for exploration purposes.
- Should create lag features with `create_lag_feature()` from Lecture 19 as indicated above.

In [20]:
print(df_train.columns)
display(df_train.info())

Index(['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770',
       'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year',
       'region', 'AveragePriceNextWeek'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 15441 entries, 0 to 18222
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  15441 non-null  datetime64[ns]
 1   AveragePrice          15441 non-null  float64       
 2   Total Volume          15441 non-null  float64       
 3   4046                  15441 non-null  float64       
 4   4225                  15441 non-null  float64       
 5   4770                  15441 non-null  float64       
 6   Total Bags            15441 non-null  float64       
 7   Small Bags            15441 non-null  float64       
 8   Large Bags            15441 non-null  float64       
 9   XLarge Bags           15441 non-null  

None

In [21]:
# Similar to RainTomorrow variable in Lecture 19 
numeric_features = ['AveragePrice', 'Total Volume', '4046', '4225', '4770','Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags',]
categorical_features = ['type', 'year', 'region',]
drop_features = ["Date"]
target = ["AveragePriceNextWeek"]

In [22]:
# Adapted from Lecture 19 

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

def preprocess_features(
    df_train,
    df_test,
    numeric_features,
    categorical_features,
    drop_features,
    target
):

    all_features = set(numeric_features + categorical_features + drop_features + target)
    if set(df_train.columns) != all_features:
        print("Missing columns", set(df_train.columns) - all_features)
        print("Extra columns", all_features - set(df_train.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
    categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
    )

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(df_train)
    ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(
        preprocessor.transform(df_train), index=df_train.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(df_test), index=df_test.index, columns=new_columns
    )

    y_train = df_train["AveragePriceNextWeek"]
    y_test = df_test["AveragePriceNextWeek"]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [23]:
# Adapted from Lecture 19
def score_lr_print_coeff(preprocessor, df_train, y_train, df_test, y_test, X_train_enc):
    lr_pipe = make_pipeline(preprocessor, Ridge())
    lr_pipe.fit(df_train, y_train)
    print("Train score: {:.4f}".format(lr_pipe.score(df_train, y_train)))
    print("Test score: {:.4f}".format(lr_pipe.score(df_test, y_test)))
    lr_coef = pd.DataFrame(
        data=lr_pipe.named_steps["ridge"].coef_.flatten(),
        index=X_train_enc.columns,
        columns=["Coef"],
    )
    return lr_coef.sort_values(by="Coef", ascending=False)

In [24]:
# Adapted from Lecture 19 
# OPTION 1: Time as a number

first_day = df_train["Date"].min()
df_train_1 = df_train.assign(
    Days_since=df_train["Date"].apply(lambda x: (x - first_day).days)
)
df_test_1 = df_test.assign(
    Days_since=df_test["Date"].apply(lambda x: (x - first_day).days)
)
# display(df_train_1.head())

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_1,
    df_test_1,
    numeric_features + ["Days_since"],
    categorical_features,
    drop_features + ["year"],
    target
)
score_lr_print_coeff(preprocessor, df_train_1, y_train, df_test_1, y_test, X_train_enc)

Train score: 0.8468
Test score: 0.7516


,Coef
AveragePrice,0.319366
region_SanFrancisco,0.097591
region_HartfordSpringfield,0.095290
region_NewYork,0.075457
region_Philadelphia,0.055506
...,...
region_Denver,-0.050518
type_conventional,-0.055376
region_SouthCentral,-0.072651
region_DallasFtWorth,-0.073299


In [25]:
# Adapted from Lecture 19 
# OPTION 2: One-hot encoding of year

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features,
    categorical_features,
    drop_features,
    target
)
score_lr_print_coeff(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.8460
Test score: 0.7953


,Coef
AveragePrice,0.325180
region_SanFrancisco,0.091341
region_HartfordSpringfield,0.088819
region_NewYork,0.070054
type_organic,0.051681
...,...
region_Denver,-0.047878
type_conventional,-0.051681
region_SouthCentral,-0.067115
region_DallasFtWorth,-0.068354


In [26]:
# Adapted from Lecture 19 
# OPTION 3: One-hot encoding of month

df_train_3 = df_train.assign(
    Month=df_train["Date"].apply(lambda x: x.month_name())
)  # x.month_name() to get the actual string
df_test_3 = df_test.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))
df_train_3[["Date", "Month"]].sort_values(by="Month")
# display(df_train_3.head())

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_3,
    df_test_3,
    numeric_features,
    categorical_features + ["Month"],
    drop_features + ["year"],
    target
)
score_lr_print_coeff(preprocessor, df_train_3, y_train, df_test_3, y_test, X_train_enc)

Train score: 0.8504
Test score: 0.8018


,Coef
AveragePrice,0.311424
region_SanFrancisco,0.106214
region_HartfordSpringfield,0.103903
region_NewYork,0.081879
region_Philadelphia,0.060428
...,...
region_Denver,-0.055380
type_conventional,-0.060229
region_SouthCentral,-0.077472
region_DallasFtWorth,-0.079459


In [27]:
# Adapted from Lecture 19 
# OPTION 4: One-hot encoding of day of week 

df_train_4 = df_train.assign(
    DayOfWeek=df_train["Date"].apply(lambda x: x.day_name())
)  # x.day_name() to get the actual string
df_test_4 = df_test.assign(DayOfWeek=df_test["Date"].apply(lambda x: x.day_name()))
df_train_4[["Date", "DayOfWeek"]].sort_values(by="DayOfWeek")
# display(df_train_4.head())

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_4,
    df_test_4,
    numeric_features,
    categorical_features + ["DayOfWeek"],
    drop_features + ["year"],
    target
)
score_lr_print_coeff(preprocessor, df_train_4, y_train, df_test_4, y_test, X_train_enc)

Train score: 0.8460
Test score: 0.7953


,Coef
AveragePrice,0.325180
region_SanFrancisco,0.091341
region_HartfordSpringfield,0.088819
region_NewYork,0.070054
type_organic,0.051681
...,...
region_Denver,-0.047878
type_conventional,-0.051681
region_SouthCentral,-0.067115
region_DallasFtWorth,-0.068354


All cases provide a train $R^2$ score of approximately 0.85 (rounded by 2 d.p.), which is greater than the baseline $R^2$ score of 0.8285800937261841. The first options of treating the time as a numerical variable by number of days provide a lower test $R^2$ score of 0.7516, lower than the baseline test $R^2$ score of 0.7631780188583048. The second, third and fourth options of applying one-hot encoding for year, month, and day of the week respectively, provide a reasonable test $R^2$ score of 0.7953, 0.8018, and 0.7953 respectively. Overall, one-hot encoding methods for months provide more effective date encoding than the other options.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: very short answer questions

Each question is worth 2 points.

<!-- BEGIN QUESTION -->

### 2.1 Time series

_Points:_ 4

The following questions pertain to Lecture 19 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    
</div>

(1) 

A common example is a business that is closed on weekends and holidays, so they wouldn't be recording data on weekends or holidays and create gaps between Mondays and Fridays, among other interspersed dates due to holidays.

(2) 

Encoding of the date would mitigate any inconsistencies with unequally spaced time points, by allowing the intervals to become more regularised. Lagged features will struggle more because it relies on the assumption that the time between a data point and its previous/next point(s) are equal for every time point, and that conversely there is a value present for every point at those fixed intervals. If certain time points are missing, then the missing time points might have to be imputed, which could introduce bias. If the time between points are inconsistent, there will simply be less time for the target values to change between some points compared to other points, which will inflate the variance.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis

_Points:_ 6

The following questions pertain to Lecture 20 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2
    
</div>

(1) 

The customers labeled "not churned" have censored churn times, and thus there are no correct target values to train and test the model, which will negatively impact the effectiveness of the model. You won't know when or <i>if</i> the customers that are "not churned" will churn in the future. This will bias your predictors towards predicting more recent customers as not churned or will not churn.

(2) 

We do not have information to answer, because just two data points, neither of which have ever left the service, are not enough to determine when one of them will do something that hasn't even been observed in the data yet. We have zero indicators about the length of time customers stay with a service, let alone other potentially helpful information such as the type of service, demographics of the customer, how they perceive the service, or how badly they need the service. The patterns in the amount of time people will spend with say, an internet service provider is not comparable to the patterns in the amount of time people will hire a construction firm or a repair service, not to mention other bottlenecks such as free trial lengths, or increases in fees depending on time.

(3) 

The survival function value is very close to 1, indicating that the customer is extremely unlikely to leave.

SOLUTION: If the survival function remains flat over a certain period the risk of churn is relatively consistent for that period of time. In most cases, when the survival function remains flat, its value is likely to be very close to 1, indicating that the customer is extremely unlikely to leave.

<!-- END QUESTION -->

<br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using PrairieLearn.
4. Make sure that the plots and output are rendered properly in your submitted file.